In [31]:
from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np, pandas as pd
import seaborn as sns
from lxml import etree

In [68]:
pages = np.arange(1, 100, 50) 
headers = {'Accept-Language' : 'en-US,en; q=0.8'}

titles = []
years = []
ratings = []
genres = []
runtimes = []
imdb_ratings = []
imdb_ratings_standardized = []
metascores = []
votes = []


for page in pages:
    response = get("https://www.imdb.com/search/title?genres=sci-fi&"+"start"+str(page)+ "&explore=title_type, genres&ref = adv_prv", headers=headers)
    
    sleep(randint(8,15))
    
    if response.status_code != 200:
       warn('Request: {}; Status code: {}'.format(requests, response.status_code))
             
    page_html = BeautifulSoup(response.text, 'html.parser')
    #print(page_html.prettify())
    dom = etree.HTML(str(page_html))
        
    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    
    for container in movie_containers:
        title = container.h3.a.text
        titles.append(title)
        if container.h3.find('span', class_ = 'lister-item-year text-muted unbold') is not None:
            year = container.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
            years.append(year)
        else:
            years.append(None)
            
        if container.p.find('span', class_ = 'certificate') is not None:
            rating = container.p.find('span', class_ = 'certificate').text 
            ratings.append(rating)
        else:
            ratings.append("")
        
        if container.p.find('span', class_ = 'genre') is not None:
            genre  = container.p.find('span', class_ = 'genre').text.replace('\n', "").rstrip().split(',')
            genres.append(genre)
        else:
            genres.append("")
        
        if container.p.find('span', class_ = 'runtime') is not None:
            time = int(container.p.find('span', class_ = 'runtime').text.replace(" min", ""))
            runtimes.append(time)
        else:
            runtimes.append(None)
            
        if container.strong is not None:
            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)
        else:
            imdb_ratings.append(None) 
        
            
        if container.p.find('span', class_ = 'metascore') is not None: 
            m_score = int(container.p.find('span', class_ = 'metascore').text)
            metascores.append(m_score)
        else:
            metascores.append(None)
            
        if container.find('span', {'name':'nv'}).get_text(strip=True) is not None: 
            vote = int(container.find('span', {'name':'nv'}).get_text(strip=True).replace(',',''))
            print(vote)
            votes.append(vote)
        else:
            votes.append(None)

86342
54044
131615
1133532
68211
494039
454405
82796
171019
91846
3108
85476
39424
186423
17502
122025
239248
57238
39439
416529
311640
363702
27439
132220
46823
494039
711845
37488
221587
546114
531898
164777
11346
90135
7376
223086
25666
222252
603223
230063
430962


AttributeError: 'NoneType' object has no attribute 'get_text'

In [41]:
print(len(titles))
print(len(years))
print(len(ratings))
print(len(genres))
print(len(runtimes))
print(len(imdb_ratings))
print(len(metascores))
print(len(votes))

100
100
100
100
100
100
100
100


In [42]:
sci_fi_df = pd.DataFrame({'movie': titles, 'year': years, 'rating': ratings, 'genre': genres,'runtime_min': runtimes, 'imdb': imdb_ratings, 'metascore': metascores, 'votes': votes})
sci_fi_df['year'] = sci_fi_df['year'].str.replace("[()I– ]", "").str[0:4].astype(int)
sci_fi_df.head(50)

C:\Users\Adrien\AppData\Local\Temp\ipykernel_10160\1676851152.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  sci_fi_df['year'] = sci_fi_df['year'].str.replace("[()I– ]", "").str[0:4].astype(int)


,movie,year,rating,genre,runtime_min,imdb,metascore,votes
0,The Sandman,2022,TV-MA,"[Drama, Fantasy, Horror]",45.0,7.8,None,86160
1,She-Hulk: Attorney at Law,2022,TV-14,"[Action, Adventure, Comedy]",NaN,5.2,None,86160
2,Prey,2022,R,"[Action, Adventure, Drama]",99.0,7.2,None,86160
3,Stranger Things,2016,TV-14,"[Drama, Fantasy, Horror]",51.0,8.7,None,86160
4,Nope,2022,R,"[Horror, Mystery, Sci-Fi]",130.0,7.1,None,86160
5,Westworld,2016,TV-MA,"[Drama, Mystery, Sci-Fi]",62.0,8.5,None,86160
6,The Boys,2019,TV-MA,"[Action, Crime, Drama]",60.0,8.7,None,86160
7,The Orville,2017,TV-14,"[Adventure, Comedy, Drama]",44.0,8.0,None,86160
8,Thor: Love and Thunder,2022,PG-13,"[Action, Adventure, Comedy]",118.0,6.7,None,86160
9,Severance,2022,TV-MA,"[Drama, Mystery, Sci-Fi]",55.0,8.7,None,86160


In [35]:
print(sci_fi_df['year'])

0     2022
1     2022
2     2022
3     2016
4     2022
      ... 
95    1990
96    2022
97    2021
98    2022
99    2014
Name: year, Length: 100, dtype: int32
